<a href="https://colab.research.google.com/github/osmartinez/modelizacion-placa-calor/blob/master/placa_calor_jacobi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Método de Jacobi. Propósito general**

In [0]:
import numpy as np
import math

In [0]:
'''
Función que descompone A
D + R = A
'''
def descomponer_matriz(A):
  p,q = A.shape
  D = np.zeros(shape=(p,q))
  R= np.zeros(shape=(p,q))
  
  for i in range(p):
    for j in range(q):
      if i == j:
        D[i][j] = A[i][j]
        R[i][j] = 0
      else:
        D[i][j] = 0
        R[i][j] = A[i][j]
  
  return D,R

In [0]:
'''
Función que invierte una matriz diagonal D
'''
def invertir_diagonal(D):
  p,q = D.shape
  D_inv = np.zeros(shape=(p,q))
  
  for i in range(p):
    for j in range (q):
      if i == j:
        D_inv[i][j] = 1/D[i][j]
        
  return D_inv

In [0]:
'''
Aplica método de Jacobi al sistema de ecuaciones Ax=b,
descomponiendo A en R+D
Utiliza como criterio de parada un número N de iteraciones
'''
def do_jacobi_secuencial_descomp_n(A, b, N=100):
  dim_b = b.shape
  dim_A = A.shape
  p,q = dim_A
  
  if(dim_b[0] != dim_A[0]):
    raise Exception('Dimensiones no compatibles')
  
  D,R = descomponer_matriz(A)
  D_inv = invertir_diagonal(D)
  
  sol = np.zeros(shape=dim_b)
  
  # itero N veces el método
  for n in range(N):
    sol = np.dot(D_inv, (b- np.dot(R,sol)) )
  
  return sol

In [0]:
'''
Aplica método de Jacobi secuencialmente al sistema Ax= b,
sin descomposición
Utiliza como criterio de parada N iteraciones
'''
def do_jacobi_secuencial_n(A, b, N=100):
  p,q = A.shape
  dim_b = b.shape
  
  if(dim_b[0] != p):
    raise Exception('Dimensiones no compatibles')
    
  sol = np.zeros(shape = dim_b)
  
  for n in range(N):
    for i in range (p):
      elemento_diag= A[i][i]
      numerador = b[i]
      for j in range(q):
        if j == i:
          continue
        numerador-=A[i][j]*sol[j]
      sol[i] = numerador/elemento_diag
  return sol

In [0]:
'''
Aplica método de Jacobi secuencialmente al sistema Ax= b,
sin descomposición
Utiliza como criterio de parada N iteraciones o tolerancia al error
'''
def do_jacobi_secuencial_n_tol(A, b, N=100, tol=1e-10):
  p,q = A.shape
  dim_b = b.shape
  
  if(dim_b[0] != p):
    raise Exception('Dimensiones no compatibles')
    
  sol = np.zeros(shape = dim_b)
  sol_actual = np.zeros(shape = dim_b)
  
  for n in range(N):
    for i in range (p):
      elemento_diag= A[i][i]
      numerador = b[i]
      for j in range(q):
        if j == i:
          continue
        numerador-=A[i][j]*sol[j]
      sol_actual[i] = numerador/elemento_diag
      
    suma =0
    for i in range (p):
      suma += math.sqrt(math.pow(sol_actual[i]-sol[i],2))
      sol[i] = sol_actual[i]
    
    if suma<tol:
      print('Tolerancia alcanzada! Iteración:'+str(n))
      break      
      
  return sol

In [134]:
A = np.array([[5,2,1,1],
              [2,6,2,1],
              [1,2,7,1],
              [1,1,2,8]])

b = np.array([29,31,26,19])

print (do_jacobi_secuencial_n_tol(A,b,100,1e-11))
print (do_jacobi_secuencial_descomp_n(A,b,100))
print (do_jacobi_secuencial_n(A,b,100))

Tolerancia alcanzada! Iteración:75
[3.99275362 2.95410628 2.16183575 0.96618357]
[3.99275362 2.95410628 2.16183575 0.96618357]
[3.99275362 2.95410628 2.16183575 0.96618357]


## **Método de Jacobi. Adaptación al problema de la placa de calor**



A continuación se desarrolla el código necesario para la aplicación del método (secuencial) de Jacobi al problema de la placa de calor.







In [0]:
'''
Definición de constantes.
Se definen las temperaturas de los extremos de la placa
'''
t_izq = 2.0
t_ab = 1.0
t_dcha = 0.0
t_ar = 0.0


In [0]:
'''
Función que dice si una matriz es estrictamente diagonal dominante
'''
def es_diagonal_dominante(A):
    abs_A = np.abs(A)
    return np.all( 2*np.diag(abs_A) >= np.sum(abs_A, axis=1) )

In [0]:
'''
Creamos la matriz A en Ax=b
p y q son las dimensiones de la placa de calor
'''
def crear_sistema(p,q):
  A = np.zeros(shape=(p*q,q*p))
  for i in range(p*q):
    A[i][i] = 1.0
    
  for j in range(p):
    for k in range(q):
      if j>1:
        A[(k-1)*p+j][(k-1)*p+j-1] = -1.0/4
      if j<p:
        A[(k-1)*p+j][(k-1)*p+j+1] = -1.0/4
      if k>1:
        A[(k-1)*p+j][(k-2)*p+j] = -1.0/4
      if k<q:
        A[(k-1)*p+j][k*p+j] = -1.0/4
  
  if es_diagonal_dominante(A):
    print('La matriz generada es estrictamente diagonal dominante')
    
    
  b = np.zeros(shape=(p*q))
  b[0] = t_izq/4 + t_ar/4
  b[p] = t_dcha/4 + t_ar/4
  b[(q-1)*p] = t_izq/4 + t_ab/4
  b[p*q-1] = t_dcha/4 + t_ab/4
  for j in range(2,p-1):
    b[j] = t_ar/4
    b[(q-1)*p+j] = t_ab/4
  for k in range(2,q-1):
    b[(k-1)*p+1] = t_izq/4
    b[k*p]=t_dcha/4
    
  
  vector_inv_diag = np.zeros(shape= (p*q))
  for j in range(p*q):
    vector_inv_diag[j] = 1/A[j][j]
  
  return A,b,vector_inv_diag

In [211]:
A,b,vector_inv_diag = crear_sistema(20,20)
do_jacobi_secuencial_n_err(A,b,100000)

La matriz generada es estrictamente diagonal dominante
Tolerancia alcanzada! Iteración:2041


array([0.76669475, 0.39369739, 0.39220051, 0.33026799, 0.27260998,
       0.22785169, 0.19425395, 0.16909226, 0.15026692, 0.13636387,
       0.12649027, 0.12014092, 0.11713133, 0.11760209, 0.12211713,
       0.13193372, 0.14966997, 0.18104681, 0.23990171, 0.36424781,
       0.6730816 , 1.18258904, 0.84483666, 0.65626146, 0.53232025,
       0.44454281, 0.38007186, 0.33184816, 0.29561154, 0.26869831,
       0.2494563 , 0.23694207, 0.23078231, 0.23115989, 0.23893272,
       0.25594776, 0.28569936, 0.33461555, 0.41431222, 0.54400793,
       0.74304261, 1.49182209, 1.14829565, 0.91762094, 0.75586675,
       0.63792744, 0.54964254, 0.48261697, 0.43163277, 0.39336151,
       0.36569456, 0.34738876, 0.33789594, 0.33732245, 0.34650609,
       0.36722527, 0.40256414, 0.4574038 , 0.5387237 , 0.65442909,
       0.80726674, 1.63640368, 1.33890291, 1.11005988, 0.93559838,
       0.80165767, 0.69795387, 0.61734441, 0.55494106, 0.50742041,
       0.47257167, 0.44902245, 0.43609027, 0.43372787, 0.44254